In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init
import torch.nn as nn

In [2]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

print ("Device is " + device)

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)


Device is cuda


In [3]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [4]:
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)
mnist_train

Dataset MNIST
    Number of datapoints: 60000
    Split: train
    Root Location: MNIST_data/
    Transforms (if any): ToTensor()
    Target Transforms (if any): None

In [5]:
mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)
mnist_test

Dataset MNIST
    Number of datapoints: 10000
    Split: test
    Root Location: MNIST_data/
    Transforms (if any): ToTensor()
    Target Transforms (if any): None

In [6]:
dataLoader = torch.utils.data.DataLoader(dataset=mnist_train,
                                         batch_size=batch_size,
                                         shuffle=True,
                                         drop_last=True)

In [31]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # dimension = (28, 28) * 1ch
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        # dimension = (14, 14) * 32ch
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        # dimension = (7, 7) * 64ch
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        # dimension = (3.5, 3.5) * 128 ch => (3, 3) * 128 ch (소숫점 이하 버림)
        self.fc1 = nn.Linear(3*3*128, 625)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(625, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight);
        torch.nn.init.xavier_uniform_(self.fc2.weight);
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out


In [32]:
model = CNN().to(device)

value = torch.Tensor(1, 1, 28, 28).to(device)
print (model(value).shape)

torch.Size([1, 10])


In [33]:
model

CNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=1152, out_features=625, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=625, out_features=10, bias=True)
)

In [34]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [35]:
# training
total_batch = len(dataLoader)
print ("Learing started.");

for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in dataLoader:
        # X => Image, Y => Label
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        
        hypothesis = model(X)
        
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print ('[Epoch: {}] cost = {}'.format(epoch+1, avg_cost))
    
print ('Learning finished')

Learing started.
[Epoch: 1] cost = 0.1772514283657074
[Epoch: 2] cost = 0.04501855745911598
[Epoch: 3] cost = 0.03123079054057598
[Epoch: 4] cost = 0.02434610016644001
[Epoch: 5] cost = 0.016777509823441505
[Epoch: 6] cost = 0.016400212422013283
[Epoch: 7] cost = 0.013715575449168682
[Epoch: 8] cost = 0.010934176854789257
[Epoch: 9] cost = 0.00968080759048462
[Epoch: 10] cost = 0.009256050921976566
[Epoch: 11] cost = 0.007574994117021561
[Epoch: 12] cost = 0.008414650335907936
[Epoch: 13] cost = 0.005602968391031027
[Epoch: 14] cost = 0.005808047950267792
[Epoch: 15] cost = 0.005699639208614826
Learning finished


In [36]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print("Accuracy:", accuracy.item())

Accuracy: 0.9866999983787537
